# Modelos

Este módulo foi dedicado ao modelo, tentamos com o Naive Bayers, o KNN, Floresta Randomica, Gradient Boosting e o Voting Classifier


Carregamos os dados e dividimos as classes em X e y, colocadas dentro de um numpy.array, para serem entradas dos modelos.

O Naive Bayes, foi escrito em um código rápido para uma tentativa com a competição, o resultado é foi inicialmente um score de 0.14 porque esquecemos de reconverter a saída. Após reconvertermos conseguimos 0.69906, todavia os dados como podemos observar nos gráficos eles não são normais, e também não normalizamos, decidimos descartar e refazer todos os modelos. 

De inicio pensamos em remover o TUE, CH2O e o FAF, testamos as submissões com as alterações também, todavia não houve melhoras.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [2]:
df = pd.read_pickle('df_transformado.pkl')
df_test = pd.read_pickle('df_test_transformado.pkl')


In [3]:
df.head()

,__Gender_Female,__Gender_Male,id,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad
0,0.0,1.0,0,24,1.699998,81.66995,1,1,2,3,1,0,2.763573,1,0.0,0.976473,1,3
1,1.0,0.0,1,18,1.56,57.0,1,1,2,3,2,0,2.0,1,1.0,1.0,0,1
2,1.0,0.0,2,18,1.71146,50.165754,1,1,2,1,1,0,1.910378,1,0.866045,1.673584,0,0
3,1.0,0.0,3,21,1.71073,131.274851,1,1,3,3,1,0,1.674061,1,1.467863,0.780199,1,6
4,0.0,1.0,4,32,1.914186,93.798055,1,1,3,2,1,0,1.979848,1,1.967973,0.931721,1,3


In [4]:
y = df['NObeyesdad']
id = df['id']
X = df.drop(['NObeyesdad','id'], axis=1)

y = y.to_numpy()
X = X.to_numpy()

id_test = df_test['id']
X_test = df_test.drop(['id'],axis=1).to_numpy()

In [5]:
nobsedy_invertio = {3: 'Overweight_Level_II', 1: 'Normal_Weight', 0: 'Insufficient_Weight',
                     6: 'Obesity_Type_III', 5: 'Obesity_Type_II', 2: 'Overweight_Level_I', 4: 'Obesity_Type_I'}


## KNN

Escolhemos o KNN por ser um clássico, fácil comprensão, usamos o kfold para avaliar as metricas ao redor de cada conjunto do modelo

In [6]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = KNeighborsClassifier(n_neighbors=10)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))



c:\Users\larae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\larae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


fold 0> accuracy =  0.8559730250481695
fold 1> accuracy =  0.8526011560693642
fold 2> accuracy =  0.8482658959537572
fold 3> accuracy =  0.8554913294797688
fold 4> accuracy =  0.8434489402697495
fold 5> accuracy =  0.8453757225433526
fold 6> accuracy =  0.8574181117533719
fold 7> accuracy =  0.8492292870905588
fold 8> accuracy =  0.860722891566265
fold 9> accuracy =  0.8578313253012049
Accuracy Mean: 0.853, Standard Deviation: 0.005


Observamos que ele tem uma queda de acuracia do fold 2, no fold 4 e no fold 7

## Floresta de Árvore de Decisão

In [8]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9070327552986512
fold 1> accuracy =  0.8959537572254336
fold 2> accuracy =  0.8892100192678227
fold 3> accuracy =  0.8935452793834296
fold 4> accuracy =  0.8959537572254336
fold 5> accuracy =  0.9041425818882466
fold 6> accuracy =  0.9060693641618497
fold 7> accuracy =  0.8925818882466281
fold 8> accuracy =  0.9108433734939759
fold 9> accuracy =  0.896867469879518
Accuracy Mean: 0.899, Standard Deviation: 0.007


In [9]:
rf_model = RandomForestClassifier(n_estimators=300)
rf_model.fit(X,y)
y_pred = rf_model.predict(X_test)


In [10]:
df_random_forest = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_random_forest['NObeyesdad']  = df_random_forest['NObeyesdad'].map(nobsedy_invertio)

df_random_forest.to_csv('submtions/randomforest.csv', index=False, sep=',')


A floresta revelou os problemas nos mesmos folds, destancando dessa vez o fold 9, que também tinha um valor mais baixo no knn. Mas se saiu melhor para o conjunto de dados. O estimador foi definido como n=300 porque foi o melhor valor encontrado nos testes, igual ao critério, que no caso por padrão é o gini, fizemos com log_loss mas não houve mudanças significativas.

## Gradient Boost

Encontramos vasculhando a documentação do skitlern, decidimos testar os dados com outro altoritmo, escolhemos algum que nunca tinhamos testado. Tentamos com além do Gradient Boost, com a Regressão Logistica e as Maquinas de Suporte Vetoriais, todavia os algoritmos não convergiam, então decidimos manter com o Gradient Boost. O algoritmo foi testado de três formas, removendo CH2O, TUE e FAF, teve perdas pequenas no treinamento. Ao usar n_estimator=300, o treinamento teve bons resultados, todavia a acuracia caiu no teste.

O algoritmo foi colocado no Kaggle sem TUE E FAF, atingiu 0.89559, com esses 0.89776, não fizemos o teste sem CH2O. 

In [11]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9104046242774566
fold 1> accuracy =  0.901252408477842
fold 2> accuracy =  0.8921001926782274
fold 3> accuracy =  0.8988439306358381
fold 4> accuracy =  0.9051059730250481
fold 5> accuracy =  0.9065510597302505
fold 6> accuracy =  0.9075144508670521
fold 7> accuracy =  0.8945086705202312
fold 8> accuracy =  0.9074698795180723
fold 9> accuracy =  0.9069879518072289
Accuracy Mean: 0.903, Standard Deviation: 0.006


In [12]:
gb_model = GradientBoostingClassifier()
gb_model.fit(X,y)
y_pred = gb_model.predict(X_test)

In [13]:
df_gradient_boosting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_gradient_boosting['NObeyesdad']  = df_gradient_boosting['NObeyesdad'].map(nobsedy_invertio)

df_gradient_boosting.to_csv('submtions/gradient_boosting.csv', index=False, sep=',')

## Voting Classifier

O algoritmo foi usado após  olhar exemplos de aplicações da Floresta de Árvores e do Gradient Boosting. Achamos interessantes pois é um cômite de cômite. Pensamos que poderiamos usar porque como observamos a cima, temos perdas de desempenho em alguns folds, então se usamos um cômite de cômite, talvez consegueriamos resultados melhores, onde o fold se saiu melhor. Embora tenha aumentado a acurácia, não aumentou como esperavamos porque precisamos encontrar valores altos no folds de menores acurácias. Mas foi aqui que conseguimos o primeiro valor acima de 0.9 no Kaggle, no caso 0.90137.

In [14]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
	clf1 = GradientBoostingClassifier()
	clf3 = GradientBoostingClassifier(n_estimators=300)
	model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9161849710982659
fold 1> accuracy =  0.9060693641618497
fold 2> accuracy =  0.8959537572254336
fold 3> accuracy =  0.899325626204239
fold 4> accuracy =  0.905587668593449
fold 5> accuracy =  0.9089595375722543
fold 6> accuracy =  0.9137764932562621
fold 7> accuracy =  0.896917148362235
fold 8> accuracy =  0.9108433734939759
fold 9> accuracy =  0.9137349397590362
Accuracy Mean: 0.907, Standard Deviation: 0.007


In [15]:
clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
clf1 = GradientBoostingClassifier()
clf3 = GradientBoostingClassifier(n_estimators=300)
model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
model.fit(train_X, train_y)
y_pred = model.predict(X_test)

In [16]:
df_voting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_voting['NObeyesdad']  = df_voting['NObeyesdad'].map(nobsedy_invertio)

df_voting.to_csv('submtions/voting.csv', index=False, sep=',')

## Podas

Seguindo a documentação do Skitlern que nos recomendou o ExtraTreeClassifier é um classificador muito mais aleatório que o RandomForest porque ao construir cada árvore, são consideradas mais divisões aleatórias do que no caso de uma floresta aleatória tradicional. Isso pode levar a modelos mais robustos em algumas situações, especialmente quando há ruído nos dados. Considerando que os dados estão meio estranhos, vistos que os valores que deveriam ser discretos, vieram como continuos e não tem muito o que podemos fazer porque o gerador de dados sintéticos os tornou dessa maneira. 

Pensamos em usar o ExtraTreesClassifier para identificar atributos com problemas e tentar novamente remove-los.

In [17]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_



array([0.03621541, 0.06093127, 0.09084781, 0.0852309 , 0.3300857 ,
       0.04625147, 0.01701335, 0.05633342, 0.03656436, 0.03354726,
       0.00204393, 0.05335283, 0.00816784, 0.05112094, 0.05246689,
       0.03982662])

Embora não esteja tão didático acima, os atributos com menores valores foram SMOKE, SCC e FCVC. Bem distantes do que estavamos questionando anteriormente, fizemos o teste.

In [18]:
exemplo = df.drop(['id','NObeyesdad'],axis=1)
exemplo.head()

,__Gender_Female,__Gender_Male,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC
0,0.0,1.0,24,1.699998,81.66995,1,1,2,3,1,0,2.763573,1,0.0,0.976473,1
1,1.0,0.0,18,1.56,57.0,1,1,2,3,2,0,2.0,1,1.0,1.0,0
2,1.0,0.0,18,1.71146,50.165754,1,1,2,1,1,0,1.910378,1,0.866045,1.673584,0
3,1.0,0.0,21,1.71073,131.274851,1,1,3,3,1,0,1.674061,1,1.467863,0.780199,1
4,0.0,1.0,32,1.914186,93.798055,1,1,3,2,1,0,1.979848,1,1.967973,0.931721,1


In [19]:
X2 = (df.drop(['NObeyesdad','id','SMOKE','SCC','FCVC'], axis=1)).to_numpy()
X2_test = (df_test.drop(['id','SMOKE','SCC','FCVC'], axis=1)).to_numpy()

# RandomForest 2 e GrandientBoost 2

Não testamos com o KNN acreditamos que a forma como estão os dados, no momento favorece os comites em relação ao classico KNN.

In [20]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X2, y)):
	# get data
	train_X, test_X = X2[train], X2[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9070327552986512
fold 1> accuracy =  0.8892100192678227
fold 2> accuracy =  0.8921001926782274
fold 3> accuracy =  0.8892100192678227
fold 4> accuracy =  0.8921001926782274
fold 5> accuracy =  0.899325626204239
fold 6> accuracy =  0.9026974951830443
fold 7> accuracy =  0.8872832369942196
fold 8> accuracy =  0.9040963855421686
fold 9> accuracy =  0.8987951807228916
Accuracy Mean: 0.896, Standard Deviation: 0.007


In [21]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X2, y)):
	# get data
	train_X, test_X = X2[train], X2[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9104046242774566
fold 1> accuracy =  0.901252408477842
fold 2> accuracy =  0.8848747591522158
fold 3> accuracy =  0.8978805394990366
fold 4> accuracy =  0.899325626204239
fold 5> accuracy =  0.9007707129094412
fold 6> accuracy =  0.9075144508670521
fold 7> accuracy =  0.890655105973025
fold 8> accuracy =  0.9118072289156627
fold 9> accuracy =  0.9074698795180723
Accuracy Mean: 0.901, Standard Deviation: 0.008


In [22]:
gb_model = GradientBoostingClassifier()
gb_model.fit(X2,y)
y_pred = gb_model.predict(X2_test)

In [23]:
df_gradient_boosting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_gradient_boosting['NObeyesdad']  = df_gradient_boosting['NObeyesdad'].map(nobsedy_invertio)

df_gradient_boosting.to_csv('submtions/gradient_boosting2.csv', index=False, sep=',')

Não houve uma melhora no treino, pelo contrário, piorou. Subimos pensando inocentemente que teria uma resposta melhor no teste. Então enviamos o Gradient Boosting que já estava tendo resultados melhores e conseguimos 0.89812, e foi melhor mesmo que os outros GradientBoosting!

## Voting Classifier 2

In [24]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X2, y)):
	# get data
	train_X, test_X = X2[train], X2[test]
	train_y, test_y = y[train], y[test]
	# fit model
	clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
	clf1 = GradientBoostingClassifier()
	clf3 = GradientBoostingClassifier(n_estimators=300)
	model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9104046242774566
fold 1> accuracy =  0.903179190751445
fold 2> accuracy =  0.8896917148362236
fold 3> accuracy =  0.8998073217726397
fold 4> accuracy =  0.901252408477842
fold 5> accuracy =  0.9046242774566474
fold 6> accuracy =  0.9104046242774566
fold 7> accuracy =  0.8925818882466281
fold 8> accuracy =  0.9151807228915663
fold 9> accuracy =  0.9074698795180723
Accuracy Mean: 0.903, Standard Deviation: 0.008


In [25]:
clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
clf1 = GradientBoostingClassifier()
clf3 = GradientBoostingClassifier(n_estimators=300)
model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
model.fit(X2, y)
y_pred = model.predict(X2_test)

In [ ]:
df_voting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_voting['NObeyesdad']  = df_voting['NObeyesdad'].map(nobsedy_invertio)

df_voting.to_csv('submtions/voting2.csv', index=False, sep=',')

Após os resultados do modelo anterior, subimos esse também, mesmo tendo valores menores no treino. Mas não se saiu melhor que o utlimo algoritmo do cômite de cômite. Conseguimos 0.90065